In [ ]:
# Algorithm to detect sidewalk based on back projecting the rgb and hsv values.
import cv2
import numpy as np
import os

def sidewalk_detection(img):
    img_org = img.copy()
    img_untouch = img.copy()
    width = img.shape[1]
    height = img.shape[0]

    # Set the region immediately in front of the vehicle as the region of interest (primary assumption)
    a = int(height/2) - 130
    b = int(height/2) + 130
    c = int(width/2) + 100
    d = int(width/2) - 100
    roi = img_untouch[d:c, a:b, :]

    # Convert RGB to HSV region of interest 
    roi_hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

    # Obtain desirable RGB and HS threshold values (robust to illumination changes)
    roi_r_mean = np.mean(roi[:,:,2].ravel())
    roi_g_mean = np.mean(roi[:,:,1].ravel())
    roi_b_mean = np.mean(roi[:,:,0].ravel())
    roi_h_mean = np.mean(roi_hsv[:,:,0].ravel())
    roi_s_mean = np.mean(roi_hsv[:,:,1].ravel())

    roi_r_thresh = int(roi_r_mean - 40) 
    roi_g_thresh = int(roi_g_mean - 30) 
    roi_b_thresh = int(np.min([roi_b_mean + 60, 255])) 
    roi_s_thresh = int(np.min([roi_s_mean + 50, 255])) 

    # Remove Noise and obtain HSV of the original image
    img = cv2.medianBlur(img, 7)
    img = cv2.GaussianBlur(img, (9,9), 0)
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Apply the threshold levels above to the whole image
    img_org[img_org[:,:,0] > roi_b_thresh] = 0
    img_org[img_hsv[:,:,1] > roi_s_thresh] = 0
    img_org[img[:,:,2] < roi_r_thresh] = 0
    img_org[img[:,:,1] < roi_g_thresh] = 0
    img_org[img_org[:,:,0] > 0] = 255 

    # Define a kernel to smoothen the thresholded image
    kernel = np.ones((7, 7), dtype=np.uint8)

    # Obtain a uniform threshold by constricting and dilating the thresholded image
    mask_erode = cv2.erode(img_org[:,:,2], kernel, iterations=7)
    mask_dilate = cv2.dilate(mask_erode, kernel, iterations=8)

    # Find the largest contour (sidewalk) in the mask
    (contours, _) = cv2.findContours(mask_dilate.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    if (len(contours) > 0):
        c_max = max(contours, key=cv2.contourArea)
        M = cv2.moments(c_max) 
        peri = cv2.arcLength(c_max, True)
        approx = cv2.approxPolyDP(c_max, 0.005 * peri, True)
        cv2.drawContours(img_untouch, [approx], -1, (0, 0, 255), -1)

    return img_untouch

Run the below cell to use camera:

In [ ]:
import cv2
import depthai as dai
import numpy as np

def get_frame(queue):
    # Get frame from queue
    frame = queue.get()
    # Convert frame to OpenCV format and return
    return frame.getCvFrame()

def get_color_camera(pipeline):
    # Configure color camera
    color = pipeline.createColorCamera()
    
    # Set Camera Resolution
    color.setResolution(dai.ColorCameraProperties.SensorResolution.THE_720_P)
    color.setVideoSize(1280, 720)
    
    # Make video sharper?
    color.initialControl.setSharpness(4)     # range: 0..4, default: 1
    color.initialControl.setLumaDenoise(0)   # range: 0..4, default: 1
    color.initialControl.setChromaDenoise(0) # range: 0..4, default: 1
    
    # Get main camera
    color.setBoardSocket(dai.CameraBoardSocket.AUTO)
    
    return color

def set_window_size(width, height):
    cv2.namedWindow("video", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("video", width, height)

if __name__ == '__main__':
    
    # Define a pipeline
    dai_pipeline = dai.Pipeline()
    
    dai_pipeline.setCameraTuningBlobPath('tuning_color_ov9782_wide_fov.bin')
    
    # Set up main camera
    color_main = get_color_camera(dai_pipeline)
    
    # Set output Xlink for main camera
    xout_main = dai_pipeline.createXLinkOut()
    xout_main.setStreamName("video")
  
    # Attach cameras to output Xlink
    color_main.video.link(xout_main.input)
    
    with dai.Device(dai_pipeline) as device:
        
        set_window_size(640, 400)
        
        # Get output queues. 
        video_queue = device.getOutputQueue(name="video", maxSize=1)

        # Set display window name
        cv2.namedWindow("video")

        # Variable used to toggle between three views
        view_counter = 0
        
        # Initiate capture counter
        existing_images = [int(file.split('.')[0]) for file in os.listdir("capture") if file.endswith('.jpg')]
        if existing_images:
            capture_counter = max(existing_images) + 1
        else:
            capture_counter = 1

        while True:
            
            # Get raw frame
            raw_frame = get_frame(video_queue)
            
            # Get overlayed frame
            overlayed_frame = sidewalk_detection(raw_frame)

            # Choose the view based on the current counter value
            if view_counter == 0:
                im_out = raw_frame
            else:
                im_out = overlayed_frame

            # Display output image
            cv2.imshow("video", im_out)

            # Check for keyboard input
            key = cv2.waitKey(1)
            if key == ord('q'):
                # Quit when q is pressed
                break
            elif key == ord('t'):
                # Toggle view when t is pressed
                view_counter = (view_counter + 1) % 2
            elif key == ord('c'):
            # Capture and save images when 'c' is pressed
                frame_name = os.path.join("capture", f"{capture_counter}.jpg")
                cv2.imwrite(frame_name, im_out)
                print(f"Image {capture_counter} captured and saved.")
                capture_counter += 1

        # Release the OpenCV window and clean up resources
        cv2.destroyAllWindows()

In [2]:
import cv2
from skimage.segmentation import slic, mark_boundaries
from skimage.util import img_as_float
from skimage import io
import numpy as np

def overlay_road_boundaries(image_path):
    # Load Image as Float
    img = img_as_float(io.imread(image_path))

    # Perform SLIC
    img_seg = slic(img, n_segments=200, compactness=8, convert2lab=True, min_size_factor=0.3)

    # Convert the image to uint8 for OpenCV
    img = (img * 255).astype(np.uint8)

    # Get unique superpixel labels
    unique_labels = np.unique(img_seg)

    # Create a dictionary to store the average color for each superpixel
    average_colors = {}

    # Calculate the average color for each superpixel
    for label in unique_labels:
        mask = (img_seg == label)
        average_colors[label] = np.mean(img[mask], axis=0)

    # Create an image with superpixel regions shaded in average color
    shaded_img = np.zeros_like(img)
    for label in unique_labels:
        mask = (img_seg == label)
        shaded_img[mask] = average_colors[label]

    # Display Image with Superpixel Regions Shaded in Average Color using OpenCV
    cv2.imshow('Superpixel Regions Shaded', shaded_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Example usage
image_path = '640px-Road_in_Norway.jpg'
overlay_road_boundaries(image_path)